# Лабораторная работа №2

In [17]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [18]:
import os
os.chdir("C:/Users/вячеслав/Desktop/Lab_01/Lab_01/")

In [19]:
df = pd.read_csv('Winequality-red.csv', sep=';')

In [20]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [21]:
# смотрим на размерность матрицы данных
df.shape

(1599, 12)

In [22]:
# смотрим сбалансированы ли классы, они не сбалансированы 
df['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [23]:
df['quality'] = df['quality'].replace(to_replace=3, value=4).replace(8, 7)

In [24]:
df['quality'].value_counts()

5    681
6    638
7    217
4     63
Name: quality, dtype: int64

In [25]:
#  разделяю предикоторы и отклики
y = df['quality']
X = df.drop(columns=['quality'])

In [26]:
#  разделяю на обучающую и тестовую выборку
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 1234) 

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [28]:
# подбираю параметры
param_grid = {  
      'n_estimators': [30, 40, 50, 100, 150, 200],
      'max_depth': [4, 5, 6],
      "max_features": [1, 3, 10],
      "min_samples_split": [2, 3, 5, 7, 10],
      "bootstrap": [True, False],
      "criterion": ["gini", "entropy"]
  }

In [29]:
model2 = RandomForestClassifier(random_state=42, class_weight=None)
grid_search_res = GridSearchCV(model2, param_grid=param_grid, cv=2, error_score='raise', 
                               pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [30]:
grid_search_res.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'bootstrap': 

In [31]:
# строю предсказание модели
y_pred_train = grid_search_res.predict(X_train)
y_pred_test = grid_search_res.predict(X_test)

In [32]:
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,4,5,6,7
4,1,10,4,1
5,0,178,51,2
6,0,55,146,16
7,0,2,39,23


In [33]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,4,5,6,7
4,6,28,13,0
5,0,371,75,4
6,0,67,352,2
7,0,5,59,89


In [49]:
# строим предсказание модели
y_pred_train2 = grid_search_res.predict_proba(X_train)
y_pred_test2 = grid_search_res.predict_proba(X_test)
print(y_pred_train2)
print(y_pred_test2)

[[0.00667439 0.16381485 0.3736111  0.45589966]
 [0.02271274 0.53429689 0.405014   0.03797637]
 [0.0134843  0.93433965 0.05072247 0.00145358]
 ...
 [0.03130988 0.65057109 0.30370063 0.01441841]
 [0.01236864 0.77822714 0.18431087 0.02509335]
 [0.0148947  0.24662916 0.5492932  0.18918293]]
[[0.11358591 0.59490017 0.24519112 0.0463228 ]
 [0.03743296 0.49731837 0.42590457 0.0393441 ]
 [0.02519392 0.28171765 0.51470218 0.17838625]
 ...
 [0.02344208 0.09166475 0.41664322 0.46824995]
 [0.01057117 0.11803293 0.53941637 0.33197953]
 [0.04062271 0.66141645 0.2850608  0.01290004]]


In [35]:
y_pred_train2 = grid_search_res.predict(X_train)
y_pred_test2 = grid_search_res.predict(X_test)
print(metrics.classification_report(y_pred_test2, y_test))

              precision    recall  f1-score   support

           4       0.06      1.00      0.12         1
           5       0.77      0.73      0.75       245
           6       0.67      0.61      0.64       240
           7       0.36      0.55      0.43        42

    accuracy                           0.66       528
   macro avg       0.47      0.72      0.48       528
weighted avg       0.69      0.66      0.67       528



In [50]:
# оцениваем долю наблюдений в тестовой выборке, для которых есть класс,вероятность принадлежать которому больше 0.8
# до калибровки
k = 0
for i in y_pred_test2:
    for j in i:
        if (j>0.8):
            k += 1
dolya_do = k/len(y_pred_test2)*100

In [51]:
dolya_do

4.924242424242424

# Делаем калибровку

In [36]:
from sklearn.calibration import CalibratedClassifierCV

In [37]:
model_sigmoid = CalibratedClassifierCV(grid_search_res, cv=2, method='sigmoid')

In [38]:
model_sigmoid.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


CalibratedClassifierCV(base_estimator=GridSearchCV(cv=2, error_score='raise',
                                                   estimator=RandomForestClassifier(bootstrap=True,
                                                                                    class_weight=None,
                                                                                    criterion='gini',
                                                                                    max_depth=None,
                                                                                    max_features='auto',
                                                                                    max_leaf_nodes=None,
                                                                                    min_impurity_decrease=0.0,
                                                                                    min_impurity_split=None,
                                                                                    min

In [39]:
y_pred_train3 = model_sigmoid.predict(X_train)
y_pred_test3 = model_sigmoid.predict(X_test)
print(metrics.classification_report(y_pred_test3, y_test))

              precision    recall  f1-score   support

           4       0.00      0.00      0.00         0
           5       0.79      0.70      0.75       260
           6       0.56      0.59      0.57       205
           7       0.45      0.46      0.46        63

    accuracy                           0.63       528
   macro avg       0.45      0.44      0.44       528
weighted avg       0.66      0.63      0.64       528



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [40]:
y_pred_train3 = model_sigmoid.predict_proba(X_train)
y_pred_test3 = model_sigmoid.predict_proba(X_test)
print(y_pred_train3)
print(y_pred_test3)

[[0.01734486 0.13664891 0.3370918  0.50891444]
 [0.02550245 0.48040381 0.47138381 0.02270994]
 [0.03274053 0.85968942 0.0914526  0.01611745]
 ...
 [0.03068138 0.70047862 0.2495619  0.01927809]
 [0.02428499 0.63441385 0.29018289 0.05111826]
 [0.02196698 0.2418144  0.57849624 0.15772239]]
[[0.17329665 0.64252318 0.16069454 0.02348562]
 [0.0299405  0.58070175 0.36425554 0.02510222]
 [0.03500358 0.31378188 0.56743137 0.08378316]
 ...
 [0.02282017 0.08610045 0.39687436 0.49420502]
 [0.02024814 0.15724244 0.50840094 0.31410847]
 [0.02841559 0.6280725  0.32131056 0.02220135]]


In [41]:
# оцениваем долю наблюдений в тестовой выборке, для которых есть класс,вероятность принадлежать которому больше 0.8
# после калибровки
k = 0
for i in y_pred_test3:
    for j in i:
        if (j>0.8):
            k += 1
dolya_posle = k/len(y_pred_test3)*100

In [42]:
dolya_posle

5.681818181818182